# Face recognition

In [1]:
import cv2
import numpy as np
import os

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')  
faceCascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
if faceCascade.empty():
    print("Error: Haar cascade not loaded!")
    exit()
font = cv2.FONT_HERSHEY_SIMPLEX
names = ["", "Urvashi", "Shradha"]  
cam = cv2.VideoCapture(0)
cam.set(3, 640)  # width
cam.set(4, 480)  # height

# Minimum face size to detect
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

# -------------------------------
# Real-time face recognition loop
# -------------------------------
while True:
    ret, img = cam.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        id, confidence = recognizer.predict(gray[y:y + h, x:x + w])

        # Confidence: 0 = perfect match, higher = worse match
        if confidence < 100:
            name = names[id] if id < len(names) else "Unknown"
            confidence_text = f"{round(100 - confidence)}%"
        else:
            name = "Unknown"
            confidence_text = f"{round(100 - confidence)}%"

        cv2.putText(img, str(name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence_text), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

    cv2.imshow('Camera', img)

    k = cv2.waitKey(10) & 0xFF
    if k == 27:  # Press ESC to exit
        break

print("\n[INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()
